In [46]:
from bs4 import BeautifulSoup
import pandas as pd
import glob

Let me do some quick and dirty testing to make sure I know how I expect the ECCO XML to work

In [124]:
with open('files/sampleECCOtxt.xml', 'r') as f:
    xmldata = f.read()
    soup = BeautifulSoup(xmldata, "lxml")

In [126]:
if soup.find('locsubjecthead'):
    print('yes')

yes


In [125]:
loclist1 = []

for item in soup.find_all('locsubjecthead'):
    newitem = item.get_text().replace('\n',' ')
    loclist1.append(newitem.strip())

In [83]:
testliststring = '; '.join(loclist1)
testliststring

'Westminster (London, England) History Early works to 1800; London (England) History Early works to 1800'

In [41]:
# test dict
testdict = {}
testdict['id'] = soup.find('documentid').get_text()

In [43]:
testdict['rando'] = soup.find('pubdate').get_text()
testdict

{'id': '1299400102', 'rando': '17420101'}

In [64]:
testdf3 = pd.DataFrame([testdict])

In [131]:
testdf3

,id,rando
0,1299400102,17420101


Okay, let's give a shot looking at OCR

In [95]:

filelist = glob.glob('D:/HistAndGeo/XML/*.xml')
len(filelist)

3384

In [96]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [107]:
filelistsegment = list(chunks(filelist, 100))

In [108]:
len(filelistsegment)

34

In [113]:
# I'm going to segment these, because I don't know how long they'll take...
# this is kind of messy, but eh.
filelistPart1 = filelistsegment[0:6]

filelistPart1 worked fine in the cell a few below where I run everything, so I then tried some bigger chunks -

In [121]:
filelistPart2 = filelistsegment[6:12]

In [130]:
filelistPart3 = filelistsegment[12:]

In [115]:
# this list of dicts is holding all the dicts for the files
# that have LoC travel-ish subject headings
listofdicts = []

In [127]:
# note, no * needed for these, it searches the substrings
termlist = ['travel', 'discov', 'explor', 'voyage', 'guide', 'antiquit']

metadata = ['documentid', 'fulltitle', 'module', 'estcid', 'imprintpublisher', 'pubdate',\
           'currentvolume', 'author', 'publicationplace', 'publicationplace']

for filesegment in filelistPart2:
    for file in filesegment:
        with open(file, 'r') as f:
            xmldata = f.read()
            soup = BeautifulSoup(xmldata, "lxml")

        # we only want information if the xml has a relevant loc subject heading
        # make sure to lower the subject heading text to remove caps
        # (note that on import, the caps in the tags are lowered anyway)
        if soup.find('locsubjecthead'):
            if any(word in soup.find('locsubjecthead').get_text().lower() for word in termlist):

                # then we can make a dictionary that contains the various metadata
                filedict = {}

                # and assign the different values.
                # note that many of the values aren't there!
                for tag in metadata:
                    if soup.find(tag):
                        filedict[tag] = soup.find(tag).get_text()


                # and finally, assing the loc sh text info
                # stripping the messy spaces, etc.
                subjectlist = []
                for subject in soup.find_all('locsubjecthead'):
                    subjectclean = subject.get_text().replace('\n',' ')
                    subjectlist.append(subjectclean.strip())
                filedict['subjectheading'] = '; '.join(subjectlist)

                # finally, add this to a list of dicts        
                listofdicts.append(filedict)        


In [128]:
dictsdf = pd.DataFrame(listofdicts)
dictsdf.to_csv('files/locshbackupcsvs.csv', encoding = 'utf8')

In [129]:
dictsdf

,author,currentvolume,documentid,estcid,fulltitle,imprintpublisher,module,pubdate,publicationplace,subjectheading
0,NaN,Volume 16,1309700116,N025387,"The World displayed; Or, A curious collection ...","Printed for J. Newbery, at the Bible and Sun, ...",History and Geography,17670101,London,Voyages and travels
1,"\nWood, John\n1705?\n1754\n1705?-1754\n",0,1309700600,N031356,An essay towards a description of the city of ...,"Printed for W. Frederick, bookseller, in Bath",History and Geography,17420101,Bath,Bath (England) Description and travel Early wo...
2,NaN,0,1309900200,N029227,"Interesting account of the early voyages, made...","Printed for the proprietors, and sold at Stalk...",History and Geography,17900101,London,Explorers Portugal Early works to 1800; Explor...
3,"\nPaterson, Samuel\n1728\n1802\n1728-1802\n",Volume 1,1309900301,N031489,An entertaining journey to the Netherlands; co...,printed for W. Smith,History and Geography,17820101,London,Netherlands Description and travel Early works...
4,"\nPaterson, Samuel\n1728\n1802\n1728-1802\n",Volume 2,1309900302,N031489,An entertaining journey to the Netherlands; co...,printed for W. Smith,History and Geography,17820101,London,Netherlands Description and travel Early works...
5,"\nPaterson, Samuel\n1728\n1802\n1728-1802\n",Volume 3,1309900303,N031489,An entertaining journey to the Netherlands; co...,printed for W. Smith,History and Geography,17820101,London,Netherlands Description and travel Early works...
6,"\nMavor, William Fordyce\n1758-1837\n",0,1309901100,T188570,"A New collection of voyages and travels, dedic...","Printed for E. Newbery, the corner of St. Paul...",History and Geography,17960101,London,Voyages and travels Early works to 1800
7,NaN,0,1310300100,T164779,A Companion to all the principal places of cur...,"Printed by and for J. Drew, Fetter-lane: sold ...",History and Geography,18000101,London,London (England) Guidebooks Early works to 1800
8,"\nStevens, Sacheverell\n",0,1310300400,T172346,"Miscellaneous remarks made on the spot, in a l...","Printed for S. Hooper, at Gay's Head, near Bea...",History and Geography,17560101,London,Italy Description and travel Early works to 1800
9,"\nPöllnitz, Karl Ludwig, Freiherr von\n1692\n1...",Volume 1,1313200301,T220401,"The memoirs of Charles-Lewis, Baron de Pollnit...","Printed for Daniel Browne, at the Black Swan, ...",History and Geography,17390101,London,Europe Description and travel
